<center>Национальный исследовательский университет ИТМО<br/>Факультет информационных технологий и программирования<br/>Прикладная математика и информатика</center>

## <center>Математическая статистика</center>
### <center>Отчёт по лабораторной работе №5</center>

<div style="text-align: right"><b>Работу выполнили:</b><br/>Белоцерковченко Андрей M3337<br/>Смирнов Андрей M3337<br/</div>

<center>Санкт-Петербург<br/>2023</center>

## Вариант 3

### Задача 1

In [ ]:
print("Hello, world!")

### Задача 2

Проверить гипотезу о равенстве средних на каждом уровне фактора с помощью модели однофакторного дисперсионного анализа (реализовать необходимо самим)

Фактор - курит/не курит ("smoker"), выходная переменная - ИМТ ("bmi")

In [16]:
import pandas as pd
from statsmodels.formula.api import ols

df = pd.read_csv("sex_bmi_smokers.csv")
split_val = ols("bmi ~ smoker", data=df).fit()

### Гипотеза и проверка библиотечным методом:

$H_0$ - средний ИМТ равен у тех кто курит и у тех кто не курит
$H_1$ - средний ИМТ различается у курящих и не курящих


In [13]:
import statsmodels.api as sm

sm.stats.anova_lm(split_val, typ=2)

,sum_sq,df,F,PR(>F)
smoker,0.699349,1.0,0.018792,0.890985
Residual,49719.501037,1336.0,NaN,NaN


$p_value >> 0.05$, а значит можно с уверенностью сказать что $H_0$ оказалась правдивой

### Своя реализация

In [17]:
from scipy.special import betainc
import pandas as pd

def f_cdf(x, a, b):
    return betainc(a / 2, b / 2, a * x / (a * x + b))

def anova_lm(data, dependent_variable, factor_variable):
    # Разбиение данных на уровни фактора
    levels = data[factor_variable].unique()
    groups = [data[dependent_variable][data[factor_variable] == level] for level in levels]

    # Общее среднее значение
    grand_mean = data[dependent_variable].mean()

    # Вычисление сумм квадратов между группами
    ssb = sum([len(group) * (group.mean() - grand_mean)**2 for group in groups])

    # Вычисление сумм квадратов внутри групп
    ssw = sum([(value - group.mean())**2 for group in groups for value in group])

    # Вычисление общей суммы квадратов
    sst = sum((data[dependent_variable] - grand_mean)**2)

    # Вычисление степеней свободы
    dfb = len(levels) - 1
    dfw = len(data) - len(levels)
    dft = len(data) - 1

    # Вычисление средних квадратов
    msb = ssb / dfb
    msw = ssw / dfw

    # Вычисление F-статистики
    f_statistic = msb / msw

    # Вычисление p-значения
    p_value = 1 - f_cdf(f_statistic, dfb, dfw)

    return {
        'Source of Variation': ['Between Groups', 'Within Groups', 'Total'],
        'Sum of Squares': [ssb, ssw, sst],
        'Degrees of Freedom': [dfb, dfw, dft],
        'Mean Square': [msb, msw, ''],
        'F-statistic': [f_statistic, '', ''],
        'P-value': [p_value, '', '']
    }

# Применение функции anova_lm
result = anova_lm(df, dependent_variable='bmi', factor_variable='smoker')

# Вывод результатов
result_df = pd.DataFrame(result)
print(result_df)

  Source of Variation  Sum of Squares  Degrees of Freedom Mean Square  \
0      Between Groups        0.699349                   1    0.699349   
1       Within Groups    49719.501037                1336   37.215195   
2               Total    49720.200386                1337               

  F-statistic   P-value  
0    0.018792  0.890985  
1                        
2                        


Таким образом, собственноручно написанный нами метод дал результат $p_value$ такого же порядка и соответственно мы можем с уверенностью сказать что ИМТ не зависит от того курит человек или нет